In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/FYP/chest_xray_for_SMOTE>.zip -d /content/extracted_files

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


In [ ]:
# Define paths to dataset
base_dir = '/content/extracted_files/chest_xray_for_SMOTE'  # Adjust the path as necessary
train_dir = os.path.join(base_dir, '/content/extracted_files/chest_xray_for_SMOTE/train')
test_dir = os.path.join(base_dir, '/content/extracted_files/chest_xray_for_SMOTE/test')


In [ ]:
# Define the data generator for loading images without augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)  # 20% for validation


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 4187 images belonging to 2 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1045 images belonging to 2 classes.


In [ ]:
# Test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Don't shuffle test data
)

Found 624 images belonging to 2 classes.


In [ ]:
print("Training set class distribution:", train_generator.classes.sum())
print("Validation set class distribution:", validation_generator.classes.sum())
print("Test set class distribution:", test_generator.classes.sum())

In [ ]:
# Build a simple CNN model for feature extraction
base_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
])

base_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
base_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 82944)               │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,392 (75.75 KB)

 Trainable params: 19,392 (75.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Function to extract features from images
def extract_features(generator):
    features = []
    labels = []

    for batch_x, batch_y in generator:
        features_batch = base_model.predict(batch_x)
        features.extend(features_batch)
        labels.extend(batch_y)

        # Break after one epoch (since the flow_from_directory generates data indefinitely)
        if len(features) >= len(generator.filenames):
            break

    return np.array(features), np.array(labels)

In [ ]:
# Extract features from training data
X_train, y_train = extract_features(train_generator)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 579ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 839ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 604ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Apply SMOTE to balance the classes
smote = SMOTE(sampling_strategy='auto')  # Auto means balance the classes
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [ ]:
print(f"Original data shape: {X_train.shape}, Resampled data shape: {X_train_resampled.shape}")


Original data shape: (4187, 82944), Resampled data shape: (6214, 82944)


In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_resampled.shape[1],)),  # Fully connected layer
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │      10,616,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,625,281 (40.53 MB)

 Trainable params: 10,625,281 (40.53 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train_resampled, y_train_resampled,
    validation_data=(X_train, y_train),  # Validation on original data (without SMOTE)
    epochs=10,  # Adjust epochs if necessary
    batch_size=32
)


Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 53s 267ms/step - accuracy: 0.6515 - loss: 0.8618 - val_accuracy: 0.9006 - val_loss: 0.3180
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 51s 262ms/step - accuracy: 0.8223 - loss: 0.3363 - val_accuracy: 0.9651 - val_loss: 0.1253
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 83s 270ms/step - accuracy: 0.7798 - loss: 0.3637 - val_accuracy: 0.9661 - val_loss: 0.0963
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 51s 259ms/step - accuracy: 0.7705 - loss: 0.3832 - val_accuracy: 0.9494 - val_loss: 0.2042
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 51s 263ms/step - accuracy: 0.7304 - loss: 0.3987 - val_accuracy: 0.9164 - val_loss: 0.2128
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 83s 268ms/step - accuracy: 0.7212 - loss: 0.4126 - val_accuracy: 0.9604 - val_loss: 0.1596
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 81s 263ms/step - accuracy: 0.7321 - loss: 0.3885 - val_accuracy: 0.9577 - val_loss: 0.1200
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 81s 258ms/step - accuracy: 0.7332 - loss: 0

In [ ]:
# Extract features from test data
X_test, y_test = extract_features(test_generator)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 682ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 697ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step


In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.6067 - loss: 0.5265
Test Accuracy: 78.21%


In [ ]:
# Generate predictions
predictions = (model.predict(X_test) > 0.5).astype("int32")


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step


In [ ]:
# Confusion Matrix and Classification Report
print("Confusion Matrix")
print(confusion_matrix(y_test, predictions))


Confusion Matrix
[[104 130]
 [  6 384]]


In [ ]:
print("Classification Report")
print(classification_report(y_test, predictions, target_names=test_generator.class_indices.keys()))


Classification Report
              precision    recall  f1-score   support

      NORMAL       0.95      0.44      0.60       234
   PNEUMONIA       0.75      0.98      0.85       390

    accuracy                           0.78       624
   macro avg       0.85      0.71      0.73       624
weighted avg       0.82      0.78      0.76       624

